# Databearbetning med Pandas
Nu när vi har gått igenom lite av grunderna ska vi titta på lite mer avancerad funktionalitet för bland annat joinoperationer, transformationer och pivoteringar. Vi börjar som vanligt med att importera pandas och läsa in ett par dataset.

In [21]:
import pandas as pd

In [22]:
orders = pd.read_csv('./assets/data/aw_orders.csv')

In [23]:
customers = pd.read_csv('./assets/data/aw_customers.csv', encoding='ISO-8859-1')

In [24]:
products = pd.read_csv('./assets/data/aw_products.csv')

### Joins
För att joina data i Pandas finns ett antal varianter men för att göra motsvarande en databasjoin används funktionen `.merge()`. I fallet nedan vill vi joina in produktnamnen till tabellen med ordrar.

In [25]:
orders.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994
1,43659,2,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982
2,43659,3,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994
3,43659,4,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994
4,43659,5,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994


In [26]:
products.head()

,ProductID,ProductNumber,ProductName,ModelName,MakeFlag,StandardCost,ListPrice,SubCategoryID
0,680,FR-R92B-58,"HL Road Frame - Black, 58",HL Road Frame,1,1059.3100,1431.50,14
1,706,FR-R92R-58,"HL Road Frame - Red, 58",HL Road Frame,1,1059.3100,1431.50,14
2,707,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,0,13.0863,34.99,31
3,708,HL-U509,"Sport-100 Helmet, Black",Sport-100,0,13.0863,34.99,31
4,709,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,0,3.3963,9.50,23


In [27]:
merged = orders.merge(products, how='inner', on='ProductID')

Om vi väljer ut de kolumner vi är intresserade av ser vi att vi nu har de fält vi är intresserade av.

In [28]:
cols = ['OrderDate', 'CustomerID', 'ProductID', 'ProductName']

merged[cols].head()

,OrderDate,CustomerID,ProductID,ProductName
0,5/31/2011,1045,776,"Mountain-100 Black, 42"
1,5/31/2011,851,776,"Mountain-100 Black, 42"
2,5/31/2011,517,776,"Mountain-100 Black, 42"
3,5/31/2011,487,776,"Mountain-100 Black, 42"
4,5/31/2011,1881,776,"Mountain-100 Black, 42"


Merge function reference table

<img src="./assets/images/merge.png" width="600" align="left">

### Concatenation
Ett annat vanligt sätt är att addera nya rader eller nya kolumner till en befintlig dataframe. Denna operation kallas i Pandas för `.concat()`.

In [29]:
df1 = products.ix[0:20, 0:4]
df2 = products.ix[0:20, 4:7]

In [30]:
df1.head()

,ProductID,ProductNumber,ProductName,ModelName
0,680,FR-R92B-58,"HL Road Frame - Black, 58",HL Road Frame
1,706,FR-R92R-58,"HL Road Frame - Red, 58",HL Road Frame
2,707,HL-U509-R,"Sport-100 Helmet, Red",Sport-100
3,708,HL-U509,"Sport-100 Helmet, Black",Sport-100
4,709,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks


In [31]:
df2.head()

,MakeFlag,StandardCost,ListPrice
0,1,1059.3100,1431.50
1,1,1059.3100,1431.50
2,0,13.0863,34.99
3,0,13.0863,34.99
4,0,3.3963,9.50


Dataseten monteras ihop med funktionen `concat` som ligger i huvudbiblioteket pd. Notera att man måste specificera vilken axel de ska concateneras på. Pandas tar hänsyn till index så rätt rader matchas ihop.

In [32]:
pd.concat([df1, df2], axis=1).head()

,ProductID,ProductNumber,ProductName,ModelName,MakeFlag,StandardCost,ListPrice
0,680,FR-R92B-58,"HL Road Frame - Black, 58",HL Road Frame,1,1059.3100,1431.50
1,706,FR-R92R-58,"HL Road Frame - Red, 58",HL Road Frame,1,1059.3100,1431.50
2,707,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,0,13.0863,34.99
3,708,HL-U509,"Sport-100 Helmet, Black",Sport-100,0,13.0863,34.99
4,709,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,0,3.3963,9.50


Om man inte specar axel får man ett helt annat beteende. I det fallet anpassas datasetens kolumnindex med varandra istället vilket gör att datasetet blir dubbelt så långt. Däremot lagras rader som inte har data för en specifik kolumn som null-värden.

In [33]:
pd.concat([df1, df2])

,ListPrice,MakeFlag,ModelName,ProductID,ProductName,ProductNumber,StandardCost
0,NaN,NaN,HL Road Frame,680.0,"HL Road Frame - Black, 58",FR-R92B-58,NaN
1,NaN,NaN,HL Road Frame,706.0,"HL Road Frame - Red, 58",FR-R92R-58,NaN
2,NaN,NaN,Sport-100,707.0,"Sport-100 Helmet, Red",HL-U509-R,NaN
3,NaN,NaN,Sport-100,708.0,"Sport-100 Helmet, Black",HL-U509,NaN
4,NaN,NaN,Mountain Bike Socks,709.0,"Mountain Bike Socks, M",SO-B909-M,NaN
5,NaN,NaN,Mountain Bike Socks,710.0,"Mountain Bike Socks, L",SO-B909-L,NaN
6,NaN,NaN,Sport-100,711.0,"Sport-100 Helmet, Blue",HL-U509-B,NaN
7,NaN,NaN,Cycling Cap,712.0,AWC Logo Cap,CA-1098,NaN
8,NaN,NaN,Long-Sleeve Logo Jersey,713.0,"Long-Sleeve Logo Jersey, S",LJ-0192-S,NaN
9,NaN,NaN,Long-Sleeve Logo Jersey,714.0,"Long-Sleeve Logo Jersey, M",LJ-0192-M,NaN


För att se detta tydligare kan vi ta bort en rad ur det ena datasetet, sortera om datat och upprepa första operationen.

In [34]:
df2.drop(3, inplace=True)
df2.sort(ascending=False)

/home/kaalih/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  from ipykernel import kernelapp as app


,MakeFlag,StandardCost,ListPrice
20,1,187.1571,337.22
19,1,204.6251,337.22
18,1,204.6251,337.22
17,1,204.6251,337.22
16,1,868.6342,1431.50
15,1,868.6342,1431.50
14,1,868.6342,1431.50
13,1,868.6342,1431.50
12,1,868.6342,1431.50
11,0,38.4923,49.99


In [35]:
pd.concat([df1, df2], axis=1)

,ProductID,ProductNumber,ProductName,ModelName,MakeFlag,StandardCost,ListPrice
0,680,FR-R92B-58,"HL Road Frame - Black, 58",HL Road Frame,1.0,1059.3100,1431.50
1,706,FR-R92R-58,"HL Road Frame - Red, 58",HL Road Frame,1.0,1059.3100,1431.50
2,707,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,0.0,13.0863,34.99
3,708,HL-U509,"Sport-100 Helmet, Black",Sport-100,NaN,NaN,NaN
4,709,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,0.0,3.3963,9.50
5,710,SO-B909-L,"Mountain Bike Socks, L",Mountain Bike Socks,0.0,3.3963,9.50
6,711,HL-U509-B,"Sport-100 Helmet, Blue",Sport-100,0.0,13.0863,34.99
7,712,CA-1098,AWC Logo Cap,Cycling Cap,0.0,6.9223,8.99
8,713,LJ-0192-S,"Long-Sleeve Logo Jersey, S",Long-Sleeve Logo Jersey,0.0,38.4923,49.99
9,714,LJ-0192-M,"Long-Sleeve Logo Jersey, M",Long-Sleeve Logo Jersey,0.0,38.4923,49.99


Som synes matchar Pandas datasetens index innan de sätts ihop. Däremot får vi null-värden för de tre sista kolumnerna på grund av att vi tog bort den raden. Se samtliga funktioner i tabellen nedan.

<img src="./assets/images/concat.png" width="600" align="left">

### Reshaping and pivoting
Pandas har stöd för ett flertal operationer för att forma och pivotera data. Ett vanligt förekommande sätt är att arbeta med hierarkiska index och funktionerna `.stack()` och `.unstack()`.

Säg att vi exempelvis vill forma om datat till visa `OrderQty` och `LineTotal` per order med kolumner för ProductID < 720. Om vi går igenom det steg för steg så kan vi börja med att ta ut det data vi är intresserade av genom att lägga på rad - och kolumnfilter.

In [36]:
orders.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994
1,43659,2,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982
2,43659,3,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994
3,43659,4,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994
4,43659,5,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994


In [37]:
row_filter = orders['ProductID'] < 720
col_filter = ['OrderQty', 'LineTotal', 'SalesOrderID', 'ProductID']

reshape = orders.ix[row_filter, col_filter]
reshape.head()

,OrderQty,LineTotal,SalesOrderID,ProductID
7,3,86.5212,43659,714
8,1,28.8404,43659,716
9,6,34.2000,43659,709
10,2,10.3730,43659,712
11,4,80.7460,43659,711


Det vi är ute efter är att indexera datat på `SalesOrderID` och `ProductID`. Detta kan vi enkelt sätta med funktionen `.set_index` enligt nedan.

In [38]:
reshape.set_index(['SalesOrderID', 'ProductID'], inplace=True)
reshape.head()

OrderQty  LineTotal
SalesOrderID ProductID                     
43659        714               3    86.5212
             716               1    28.8404
             709               6    34.2000
             712               2    10.3730
             711               4    80.7460

Vi ha titta på vårt index och se att vi numera har ett hierarkiskt index

In [40]:
print(reshape.index.nlevels)
print(reshape.index.names)

2
['SalesOrderID', 'ProductID']


När vi har `ProductID` som index kan vi enkelt använda oss av funktionen `.unstack()` för att lägga ut valfritt index till kolumner, i detta fall vårt andra index med id 1.

In [41]:
reshape = reshape.unstack(1)
reshape.head()

OrderQty                                             ...  \
ProductID         707  708  709  710  711  712  714  715  716 717 ...   
SalesOrderID                                                      ...   
43659             NaN  NaN  6.0  NaN  4.0  2.0  3.0  NaN  1.0 NaN ...   
43661             NaN  5.0  NaN  NaN  2.0  4.0  NaN  4.0  2.0 NaN ...   
43664             NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN  1.0 NaN ...   
43665             1.0  NaN  6.0  NaN  2.0  2.0  NaN  2.0  NaN NaN ...   
43667             NaN  NaN  NaN  3.0  NaN  NaN  NaN  NaN  NaN NaN ...   

             LineTotal                                                        \
ProductID          709   710     711     712      714       715      716 717   
SalesOrderID                                                                   
43659             34.2   NaN  80.746  10.373  86.5212       NaN  28.8404 NaN   
43661              NaN   NaN  40.373  20.746      NaN  115.3616  57.6808 NaN   
43664              NaN   NaN     NaN     NaN  28.8404       NaN  28.8404 NaN   
43665             34.2   NaN  40.373  10.373      NaN   57.6808      NaN NaN   
43667              NaN  17.1     NaN     NaN      NaN       NaN      NaN NaN   

                      
ProductID    718 719  
SalesOrderID          
43659        NaN NaN  
43661        NaN NaN  
43664        NaN NaN  
43665        NaN NaN  
43667        NaN NaN  

[5 rows x 24 columns]

När vi har datat i den här formen vill vi göra samma sak med våra kolumner som nu också är hierarkiska. Vår första nivå är inte namngiven men om vi tittar på etiketterna så ser vi att det är våra mätetal. För att pivotera dessa till rader använder vi `.stack()`.

In [42]:
print(reshape.columns.nlevels)
print(reshape.columns.names)
print(reshape.columns.levels)

2
[None, 'ProductID']
[['OrderQty', 'LineTotal'], [707, 708, 709, 710, 711, 712, 714, 715, 716, 717, 718, 719]]


In [43]:
reshape.stack(0).head()

ProductID               707       708   709  710     711     712      714  \
SalesOrderID                                                                
43659        OrderQty   NaN       NaN   6.0  NaN   4.000   2.000   3.0000   
             LineTotal  NaN       NaN  34.2  NaN  80.746  10.373  86.5212   
43661        OrderQty   NaN    5.0000   NaN  NaN   2.000   4.000      NaN   
             LineTotal  NaN  100.9325   NaN  NaN  40.373  20.746      NaN   
43664        OrderQty   NaN       NaN   NaN  NaN     NaN     NaN   1.0000   

ProductID                    715      716  717  718  719  
SalesOrderID                                              
43659        OrderQty        NaN   1.0000  NaN  NaN  NaN  
             LineTotal       NaN  28.8404  NaN  NaN  NaN  
43661        OrderQty     4.0000   2.0000  NaN  NaN  NaN  
             LineTotal  115.3616  57.6808  NaN  NaN  NaN  
43664        OrderQty        NaN   1.0000  NaN  NaN  NaN

Vi kan också kedja operationer tillsammans i Python/Pandas vilket blir betydligt mer kompakt än vad vi gjort hittills.

In [44]:
reshape2 = orders.ix[row_filter, col_filter]

In [45]:
reshape2.set_index(['SalesOrderID', 'ProductID']).unstack(1).stack(0).head()

ProductID               707       708   709  710     711     712      714  \
SalesOrderID                                                                
43659        OrderQty   NaN       NaN   6.0  NaN   4.000   2.000   3.0000   
             LineTotal  NaN       NaN  34.2  NaN  80.746  10.373  86.5212   
43661        OrderQty   NaN    5.0000   NaN  NaN   2.000   4.000      NaN   
             LineTotal  NaN  100.9325   NaN  NaN  40.373  20.746      NaN   
43664        OrderQty   NaN       NaN   NaN  NaN     NaN     NaN   1.0000   

ProductID                    715      716  717  718  719  
SalesOrderID                                              
43659        OrderQty        NaN   1.0000  NaN  NaN  NaN  
             LineTotal       NaN  28.8404  NaN  NaN  NaN  
43661        OrderQty     4.0000   2.0000  NaN  NaN  NaN  
             LineTotal  115.3616  57.6808  NaN  NaN  NaN  
43664        OrderQty        NaN   1.0000  NaN  NaN  NaN

Alternativt...

In [46]:
orders.head(3)

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994
1,43659,2,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982
2,43659,3,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994


In [47]:
cols = ['EmployeeID', 'UnitPriceDiscount', 'OrderQty', 'LineTotal']

orders[cols].set_index(['EmployeeID', 'UnitPriceDiscount'])

OrderQty     LineTotal
EmployeeID UnitPriceDiscount                        
279        0.00                      1   2024.994000
           0.00                      3   6074.982000
           0.00                      1   2024.994000
           0.00                      1   2039.994000
           0.00                      1   2039.994000
           0.00                      2   4079.988000
           0.00                      1   2039.994000
           0.00                      3     86.521200
           0.00                      1     28.840400
           0.00                      6     34.200000
           0.00                      2     10.373000
           0.00                      4     80.746000
           0.00                      1    419.458900
           0.00                      1    874.794000
282        0.00                      1    809.760000
           0.00                      1    714.704300
           0.00                      2   1429.408600
           0.00                      4     20.746000
           0.00                      4    115.361600
           0.00                      2   1445.189800
           0.00                      3   6074.982000
           0.00                      2   4049.988000
           0.00                      2     40.373000
           0.00                      2   1637.400000
           0.00                      4   8099.976000
           0.00                      2   4079.988000
           0.00                      2     57.680800
           0.00                      2   4049.988000
           0.00                      5    100.932500
           0.00                      3   1258.376700
...                                ...           ...
275        0.00                      9    211.356000
           0.00                      3     72.882000
           0.00                      4    255.600000
           0.00                      3    655.362000
           0.00                      3    125.982000
           0.00                      3   2456.100000
           0.00                      4    106.896000
           0.00                      1    158.430000
           0.00                      3    125.982000
           0.00                      5    792.150000
           0.05                     15    548.546625
           0.00                      3    971.982000
           0.00                      4   5507.976000
           0.00                     10  13919.940000
           0.00                      5   6959.970000
           0.00                      2   2783.988000
           0.00                      6     72.864000
           0.00                      2    485.988000
           0.00                     10    548.940000
           0.00                      7    260.064000
           0.00                      5    242.970000
           0.00                      2     32.544000
           0.00                      4    126.336000
           0.00                      4    599.496000
           0.00                      5    749.370000
           0.00                      2    316.860000
           0.00                      1    809.760000
           0.00                      4   3274.800000
           0.00                      3     97.182000
           0.40                      3    203.396400

[60919 rows x 2 columns]

In [48]:
result = (reshape2.set_index(['SalesOrderID', 'ProductID'])
         .unstack(1)
         .stack(0)
         )

result.head()

ProductID               707       708   709  710     711     712      714  \
SalesOrderID                                                                
43659        OrderQty   NaN       NaN   6.0  NaN   4.000   2.000   3.0000   
             LineTotal  NaN       NaN  34.2  NaN  80.746  10.373  86.5212   
43661        OrderQty   NaN    5.0000   NaN  NaN   2.000   4.000      NaN   
             LineTotal  NaN  100.9325   NaN  NaN  40.373  20.746      NaN   
43664        OrderQty   NaN       NaN   NaN  NaN     NaN     NaN   1.0000   

ProductID                    715      716  717  718  719  
SalesOrderID                                              
43659        OrderQty        NaN   1.0000  NaN  NaN  NaN  
             LineTotal       NaN  28.8404  NaN  NaN  NaN  
43661        OrderQty     4.0000   2.0000  NaN  NaN  NaN  
             LineTotal  115.3616  57.6808  NaN  NaN  NaN  
43664        OrderQty        NaN   1.0000  NaN  NaN  NaN

Pandas has another function called `.pivot()` that has similar functionality.

In [49]:
orders.pivot(index='SalesOrderID', columns='ProductID', values='LineTotal').head()

ProductID,707,708,709,710,711,712,714,715,716,717,...,989,990,991,992,993,994,996,997,998,999
SalesOrderID,,,,,,,,,,,,,,,,,,,,,
43659,NaN,NaN,34.2,NaN,80.746,10.373,86.5212,NaN,28.8404,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43661,NaN,100.9325,NaN,NaN,40.373,20.746,NaN,115.3616,57.6808,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Map and Apply
Pandas har inbyggt stöd för många kolumnfunktioner men i de fall man behöver göra något utanför standardfunktionerna kan man använda sig av apply och map. 

+ Map applicerar en funktion på varje element i en kolumn
+ Apply applicerar en funktion på hela kolumnen

Ett exempel är om man vill kategorisera eller beskriva data. Enklast är antagligen att ha en färdig dimensionstabell att joina på men i avsaknad av en sådan så är det trevligt att kunna använda map. Alternativ av typen långa CASE-satser är inte så kul.

In [50]:
products.head()

,ProductID,ProductNumber,ProductName,ModelName,MakeFlag,StandardCost,ListPrice,SubCategoryID
0,680,FR-R92B-58,"HL Road Frame - Black, 58",HL Road Frame,1,1059.3100,1431.50,14
1,706,FR-R92R-58,"HL Road Frame - Red, 58",HL Road Frame,1,1059.3100,1431.50,14
2,707,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,0,13.0863,34.99,31
3,708,HL-U509,"Sport-100 Helmet, Black",Sport-100,0,13.0863,34.99,31
4,709,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,0,3.3963,9.50,23


In [51]:
category = {14 : 'Frames', 23 : 'Clothes'}

In [52]:
products['SubCategoryID'].map(category)

0       Frames
1       Frames
2          NaN
3          NaN
4      Clothes
5      Clothes
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12      Frames
13      Frames
14      Frames
15      Frames
16      Frames
17      Frames
18      Frames
19      Frames
20      Frames
21      Frames
22      Frames
23      Frames
24      Frames
25      Frames
26      Frames
27      Frames
28      Frames
29      Frames
        ...   
265        NaN
266        NaN
267        NaN
268        NaN
269        NaN
270        NaN
271        NaN
272        NaN
273        NaN
274        NaN
275        NaN
276        NaN
277        NaN
278        NaN
279        NaN
280        NaN
281        NaN
282        NaN
283        NaN
284        NaN
285        NaN
286        NaN
287        NaN
288        NaN
289        NaN
290        NaN
291        NaN
292        NaN
293        NaN
294        NaN
Name: SubCategoryID, dtype: object

In [53]:
products['SubCategoryNames'] = products['SubCategoryID'].map(category)
products.head()

,ProductID,ProductNumber,ProductName,ModelName,MakeFlag,StandardCost,ListPrice,SubCategoryID,SubCategoryNames
0,680,FR-R92B-58,"HL Road Frame - Black, 58",HL Road Frame,1,1059.3100,1431.50,14,Frames
1,706,FR-R92R-58,"HL Road Frame - Red, 58",HL Road Frame,1,1059.3100,1431.50,14,Frames
2,707,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,0,13.0863,34.99,31,NaN
3,708,HL-U509,"Sport-100 Helmet, Black",Sport-100,0,13.0863,34.99,31,NaN
4,709,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,0,3.3963,9.50,23,Clothes


Vi kan också på samma sätt applicera en valfri funktion eller en funktion som vi själva skapat. Exempelvis för att avrunda värden.

In [54]:
products['ListPrice'].map(round).head()

0    1432
1    1432
2      35
3      35
4      10
Name: ListPrice, dtype: int64

Eller skapa en egen funktion.

In [55]:
def dollars_and_cents(data):
    x = str(data)
    return x.split('.')

products['StandardCost'].map(dollars_and_cents).head()

0    [1059, 31]
1    [1059, 31]
2    [13, 0863]
3    [13, 0863]
4     [3, 3963]
Name: StandardCost, dtype: object

Som vi ser ovan så returneras en ny kolumn med en lista bestående av två element, dollar och cent. Hur gör vi om vi vill ha dessa i varsin kolumn? `.map()` arbetar elementvis eller horisontellt om man hellre vill. I det här fallet vill vi ju dela listan vertikalt och som tur är finns det en funktion för det som heter `.apply()`. 

Vill vill ha datat i varsin kolumn. Kolumner kallas ju för `Series` i Pandas. Med andra ord borde vi kunna använda `.apply(pd.Series)` och få det resultat vi vill ha?

In [56]:
products['StandardCost'].map(dollars_and_cents).apply(pd.Series).head()

,0,1
0,1059,31
1,1059,31
2,13,0863
3,13,0863
4,3,3963


Kraftfullt och relativt straight forward. Det går att komprimera detta ännu mer med en lambdafunktion om man önskar.

In [57]:
products['StandardCost'].apply(lambda x: pd.Series(str(x).split('.'))).head()

,0,1
0,1059,31
1,1059,31
2,13,0863
3,13,0863
4,3,3963


In [58]:
orders.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994
1,43659,2,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982
2,43659,3,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994
3,43659,4,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994
4,43659,5,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994


In [59]:
def demo(x):
    if x > 2:
        return 'big order'

orders['OrderQty'].map(demo)

0             None
1        big order
2             None
3             None
4             None
5             None
6             None
7        big order
8             None
9        big order
10            None
11       big order
12            None
13            None
14            None
15            None
16            None
17       big order
18       big order
19            None
20       big order
21            None
22            None
23            None
24       big order
25            None
26            None
27            None
28       big order
29       big order
           ...    
60889    big order
60890    big order
60891    big order
60892    big order
60893    big order
60894    big order
60895    big order
60896         None
60897    big order
60898    big order
60899    big order
60900    big order
60901    big order
60902    big order
60903    big order
60904         None
60905    big order
60906         None
60907    big order
60908    big order
60909    big order
60910       

In [60]:
customers['LastName'].map(str.upper)

0                   ABEL
1            ABERCROMBIE
2                ACEVEDO
3                 ACHONG
4               ACKERMAN
5                  ADAMS
6                  ADAMS
7                  ADAMS
8                  ADINA
9               AGCAOILI
10               AGUILAR
11              AHLERING
12                 AKERS
13                  ALAN
14               ALBERTS
15              ALBRIGHT
16                ALBURY
17                ALCORN
18              ALDERSON
19             ALEXANDER
20             ALEXANDER
21             ALEXANDER
22                 ALLEN
23                 ALLEN
24                 ALLEN
25               ALLISON
26              ALPUERTO
27            ALTAMIRANO
28              ALVARADO
29                ALVARO
             ...        
605               TROYER
606             TRUJILLO
607             TSOFLIAS
608                TUELL
609             TUFFIELD
610                UDDIN
611                UPPAL
612                 VACA
613               VALDEZ


Därefter kan vi enkelt concatenera med vårt orginaldataset. Hela operationen kan skrivas så här.

In [61]:
x = products['StandardCost'].apply(lambda x: pd.Series(str(x).split('.')))
x.columns = ['Dollars', 'Cents']

pd.concat([products, x], axis=1).head()

,ProductID,ProductNumber,ProductName,ModelName,MakeFlag,StandardCost,ListPrice,SubCategoryID,SubCategoryNames,Dollars,Cents
0,680,FR-R92B-58,"HL Road Frame - Black, 58",HL Road Frame,1,1059.3100,1431.50,14,Frames,1059,31
1,706,FR-R92R-58,"HL Road Frame - Red, 58",HL Road Frame,1,1059.3100,1431.50,14,Frames,1059,31
2,707,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,0,13.0863,34.99,31,NaN,13,0863
3,708,HL-U509,"Sport-100 Helmet, Black",Sport-100,0,13.0863,34.99,31,NaN,13,0863
4,709,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,0,3.3963,9.50,23,Clothes,3,3963


### Replace
Pandas har en smidig funktion för att ersätta värden som otroligt nog heter `.replace()`. Den fungerar som man kan förvänta sig men har en del mer avancerade features. 

Om vi exempelvis vill ersätta värden 1/0 i kolumnen `MakeFlag` kan vi göra det såhär.

In [63]:
print(products['MakeFlag'].replace(1, 'Ja').head())
print(products['MakeFlag'].replace(0, 'Nej').head())

0    Ja
1    Ja
2     0
3     0
4     0
Name: MakeFlag, dtype: object
0      1
1      1
2    Nej
3    Nej
4    Nej
Name: MakeFlag, dtype: object


Allra bäst är ju att ersätta båda värdena i en operation vilket vi kan genom att mata `.replace()` med listor på det vi vill ändra.

In [64]:
products['MakeFlag'].replace([0, 1], ['Nej', 'Ja']).head()

0     Ja
1     Ja
2    Nej
3    Nej
4    Nej
Name: MakeFlag, dtype: object

Det går också att skicka in en dict precis som vi gjorde när vi använde `.map()` ovan.

In [65]:
make_flag = {0 : 'Nej', 1 : 'Ja'}

products['MakeFlag'].replace(make_flag).head()

0     Ja
1     Ja
2    Nej
3    Nej
4    Nej
Name: MakeFlag, dtype: object

### Discretization and binning
I det här stycket ska vi gå igenom tekniker för att jobba med binning, kategoriska variabler mm. Vi börjar med att titta på binning och deciler. Pandas har två funktioner, `.cut()` och `.qcut()`, för dessa ändamål.

+ `.cut()` fungerar så att den skär den kontinuerliga variabeln i lika stora delar men tar inte hänsyn till om frekvensen av observationer blir snedvriden.
+ `.qcut()` tar hänsyn till frekvensen av observationer och justerar spannen på den kontinuerliga variabeln istället.

Låt oss titta på exempel på detta. Vi börjar med att kika på några varianter av `.cut()`. För att dela upp en variabel i x antal bins gör man såhär.

In [66]:
orders.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994
1,43659,2,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982
2,43659,3,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994
3,43659,4,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994
4,43659,5,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994


In [67]:
var = orders['LineTotal']

pd.cut(var, 10, include_lowest=True).value_counts()

[-26.518, 2790.598]       52315
(2790.598, 5579.823]       5667
(5579.823, 8369.0475]      1778
(8369.0475, 11158.272]      682
(11158.272, 13947.496]      277
(13947.496, 16736.721]      135
(16736.721, 19525.945]       36
(19525.945, 22315.17]        16
(22315.17, 25104.394]        10
(25104.394, 27893.619]        3
Name: LineTotal, dtype: int64

Om vi istället vill ha bättre kontroll på resultatet kan vi också ange våra egna intervall

In [68]:
bins = [0, 5000, 15000, 30000]

pd.cut(var, bins).value_counts()

(0, 5000]         57464
(5000, 15000]      3334
(15000, 30000]      121
Name: LineTotal, dtype: int64

Vi kan också namnge våra intervall för att snygga till uppställningen lite.

In [69]:
bins = [0, 5000, 15000, 30000]
names = ['0 < 5000', '5000 < 15000', '15000 < 30000']

pd.cut(var, bins, labels=names).value_counts()

0 < 5000         57464
5000 < 15000      3334
15000 < 30000      121
Name: LineTotal, dtype: int64

Om vi nu istället tittar på `.qcut()` så ser vi att vi nu får likvärdiga intervall i antal observationer istället.

In [70]:
pd.qcut(var, 10).value_counts()

(1295.988, 2039.994]     6204
(43.152, 89.988]         6192
(297.747, 461.694]       6188
[1.374, 43.152]          6115
(461.694, 809.328]       6097
(164.404, 297.747]       6095
(2039.994, 3758.352]     6066
(3758.352, 27893.619]    5998
(809.328, 1295.988]      5995
(89.988, 164.404]        5969
Name: LineTotal, dtype: int64

Vi kan också skapa våra egna etiketter om vi vill. 

In [71]:
deciles = ['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10']

pd.qcut(var, 10, deciles).value_counts()

d8     6204
d2     6192
d5     6188
d1     6115
d6     6097
d4     6095
d9     6066
d10    5998
d7     5995
d3     5969
Name: LineTotal, dtype: int64

Ett vanligt behov när man jobba med analys i Python är att konvertera textetiketter till numeriska kategorier. De två vanligaste varianterna är:

+ Koda om ickenumeriska kategorier till numeriska kategorier
+ Skapa dummyvariabler av typen 0/1

Vi börjar att titta på det första exemplet. I kolumnen `SubCategoryNames` är kategorierna beskrivna i text. Om vi vill köra en algoritm på detta kommer det inte att fungera så vi vill koda om dessa till numeriska. För att underlätta det har Pandas en datastruktur som kallas `Categorical` och används så här.

In [72]:
subcat = products['SubCategoryNames']
pd.Categorical(subcat)

[Frames, Frames, NaN, NaN, Clothes, ..., NaN, NaN, NaN, NaN, NaN]
Length: 295
Categories (2, object): [Clothes, Frames]

In [73]:
print type(pd.Categorical(subcat))

SyntaxError: invalid syntax (<ipython-input-73-66c8557593a6>, line 1)

Som vi ser ovan så skapas det ett objekt av typen Categorical. Detta kan vi använda för att koda om variabler åt endera hållet.

In [74]:
cat = pd.Categorical(subcat)

print(cat.categories)
print(cat.value_counts())

Index(['Clothes', 'Frames'], dtype='object')
Clothes     4
Frames     33
dtype: int64


Om vi tittar på attributet `.codes` så ser vi också att Pandas hanterar null-värden genom att sätta dem till -1.

In [75]:
products['codes'] = cat.codes

In [76]:
products.head()

,ProductID,ProductNumber,ProductName,ModelName,MakeFlag,StandardCost,ListPrice,SubCategoryID,SubCategoryNames,codes
0,680,FR-R92B-58,"HL Road Frame - Black, 58",HL Road Frame,1,1059.3100,1431.50,14,Frames,1
1,706,FR-R92R-58,"HL Road Frame - Red, 58",HL Road Frame,1,1059.3100,1431.50,14,Frames,1
2,707,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,0,13.0863,34.99,31,NaN,-1
3,708,HL-U509,"Sport-100 Helmet, Black",Sport-100,0,13.0863,34.99,31,NaN,-1
4,709,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,0,3.3963,9.50,23,Clothes,0


Det andra scenariot är att koda om kolumnen till dummyvariabler. Beroende på algoritm är det ibland helt nödvändigt. Pandas har en funktion som heter `.get_dummies()` vilket gör detta superenkelt.

In [77]:
subcat

0       Frames
1       Frames
2          NaN
3          NaN
4      Clothes
5      Clothes
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12      Frames
13      Frames
14      Frames
15      Frames
16      Frames
17      Frames
18      Frames
19      Frames
20      Frames
21      Frames
22      Frames
23      Frames
24      Frames
25      Frames
26      Frames
27      Frames
28      Frames
29      Frames
        ...   
265        NaN
266        NaN
267        NaN
268        NaN
269        NaN
270        NaN
271        NaN
272        NaN
273        NaN
274        NaN
275        NaN
276        NaN
277        NaN
278        NaN
279        NaN
280        NaN
281        NaN
282        NaN
283        NaN
284        NaN
285        NaN
286        NaN
287        NaN
288        NaN
289        NaN
290        NaN
291        NaN
292        NaN
293        NaN
294        NaN
Name: SubCategoryNames, dtype: object

In [78]:
pd.get_dummies(subcat).head()

,Clothes,Frames
0,0.0,1.0
1,0.0,1.0
2,0.0,0.0
3,0.0,0.0
4,1.0,0.0


Vill man även koda upp null-värden så är det bara att sätta en parameter.

In [79]:
pd.get_dummies(subcat, dummy_na=True).head()

,Clothes,Frames,nan
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0


### Case-satser

In [80]:
orders.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994
1,43659,2,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982
2,43659,3,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994
3,43659,4,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994
4,43659,5,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994


In [81]:
import numpy as np

cond = (orders['OrderQty'] > 2) & (orders['UnitPriceDiscount'] == 0)

orders['case'] = np.where(cond, 'big order', 'not so big')

In [82]:
orders.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,case
0,43659,1,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994,not so big
1,43659,2,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982,big order
2,43659,3,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994,not so big
3,43659,4,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994,not so big
4,43659,5,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994,not so big
